In [18]:
import requests
from bs4 import BeautifulSoup
# Make a request to the website
r = requests.get('https://bola.kompas.com/')
# Create an object to parse the HTML format
soup = BeautifulSoup(r.content, 'html.parser')
# Retrieve all popular news links (Fig. 1)
link = []
for i in soup.find('div', {'class':'most__wrap'}).find_all('a'):
    i['href'] = i['href'] + '?page=all'
    link.append(i['href'])
# For each link, we retrieve paragraphs from it, combine each paragraph as one string, and save it to documents (Fig. 2)
documents = []
for i in link:
    # Make a request to the link
    r = requests.get(i)
  
    # Initialize BeautifulSoup object to parse the content 
    soup = BeautifulSoup(r.content, 'html.parser')
  
    # Retrieve all paragraphs and combine it as one
    sen = []
    for i in soup.find('div', {'class':'read__content'}).find_all('p'):
        sen.append(i.text)
  
    # Add the combined paragraphs to documents
    documents.append(' '.join(sen))

In [19]:
import re
import string
documents_clean = []
for d in documents:
    # Remove Unicode
    document_test = re.sub(r'[^\x00-\x7F]+', ' ', d)
    # Remove Mentions
    document_test = re.sub(r'@\w+', '', document_test)
    # Lowercase the document
    document_test = document_test.lower()
    # Remove punctuations
    document_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)
    # Lowercase the numbers
    document_test = re.sub(r'[0-9]', '', document_test)
    # Remove the doubled space
    document_test = re.sub(r'\s{2,}', ' ', document_test)
    documents_clean.append(document_test)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
# Instantiate a TfidfVectorizer object
vectorizer = TfidfVectorizer()
# It fits the data and transform it as a vector
X = vectorizer.fit_transform(documents_clean)
# Convert the X as transposed matrix
X = X.T.toarray()
# Create a DataFrame and set the vocabulary as the index
df = pd.DataFrame(X, index=vectorizer.get_feature_names())

In [21]:
import numpy as np
def get_similar_articles(q, df):
  print("query:", q)
  print("Here are the articles with the highest  similarity values: ")
  # Convert the query become a vector
  q = [q]
  q_vec = vectorizer.transform(q).toarray().reshape(df.shape[0],)
  sim = {}
  # Calculate the similarity
  for i in range(10):
    sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)
  
  # Sort the values 
  sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)
  # Print the articles and their similarity values
  for k, v in sim_sorted:
    if v != 0.0:
      print("Similaritas:", v)
      print(documents_clean[k])
      print()
# Add The Query
q1 = input()
# Call the function
get_similar_articles(q1, df)

barcelona
query: barcelona
Here are the articles with the highest  similarity values: 
Similaritas: 0.03950152928836981
kompas com china berhasil mempertahankan singgasana perolehan medali hingga hari ke perhelatan olimpiade tokyo atau sampai selasa malam wib china menempati posisi pertama klasemen medali olimpiade tokyo dengan raihan emas perak dan perunggu di posisi kedua terdapat amerika serikat as yang sudah mengoleksi emas perak dan perunggu jumlah total perolehan medali as sebenarnya lebih banyak daripada china yakni berbanding akan tetapi china berhak menempati puncak klasemen medali olimpiade tokyo karena memiliki kepingan emas lebih banyak baca juga masuk besar klasemen olimpiade tokyo indonesia sementara penuhi target adapun di peringkat ketiga ada tuan rumah jepang dengan raihan medali emas perak dan perunggu atau total medali dapatkan informasi inspirasi dan insight di email kamu daftarkan email sementara itu indonesia kini menempati posisi ke dengan koleksi satu medali ema